In [1]:
import sys
sys.path.append('../')
import latexStrings as ls
import numpy as np
import matplotlib.pyplot as plt
from math import *
from IPython.display import Latex
import odesolver
from scipy import optimize as op
import scipy.linalg as linear

# Ejercicio 3

Tomemos el siguiente PVI: 
\begin{equation} \label{eq:3}
 \begin{array}{c} y_1' = - y_1 + y_2 \\
y_2' =- y_1 - y_2 \end{array} 
\qquad \mbox{con} \qquad \begin{array}{c}
y_1(0)=0 \\ y_2(0)=1 \end{array} \qquad \mbox{para } t\in [0,1] \qquad 
\end{equation}


con solución exacta  $\overrightarrow{y}(t)=\left(\begin{array}{c} y_1(t) \\
y_2(t)\end{array} \right)$ está dada por :
\begin{equation} \label{eq:4}
 \begin{array}{c} y_1(t) = e^{-t}\sin(t) \\
y_2(t) = e^{-t}\cos(t) \end{array}
\end{equation}

Sea $\overrightarrow{W}_j$ el vector de aprximación correspondiente a la aplicación del método del Punto Medio Implícito con paso $h_j$. La entrada $w_i$ es la aproximación en $t=1$ de $y_i$ para $i,j \in \{1,2\}$

Sea $E_{PMImpl}(h_j)=\Vert W_j-\overrightarrow{y}(1)\Vert _2$ el error global en $t=1$ para $j \in \{1,2\}$ 


$\overrightarrow{G}_j:=\vert W_j-\overrightarrow{y}(1) \vert$ $\quad$cuya entrada $g_i=\vert w_1-y_i(1) \vert$ 

Aplicaremos el método  para $h_1=0.1$ y $h_2=0.01$:

In [2]:
f = lambda t, y : np.array([-y[0]+y[1], -y[0]-y[1]])
soln = lambda t: np.array([np.exp(-T)*np.sin(T), np.exp(-T)*np.cos(T)])
y0 = np.array([0,1])
T,W = odesolver.solve(f, y0, (0,1), 10, method = 'Implicit Midpoint')
globalError1=linear.norm(W[:,10]-soln(T)[:,10])
dif=abs(W[:,10]-soln(T)[:,10]) #error global por entrada en t=1
Latex('$h_1=0.1$'+ ls.latexVector(W[:,10],'W_1',form='%f')+ ls.latexVector(dif,'G_1',form='%f') +' $E_{PMImpl}(h_1) = '+str(globalError1)+'$')

<IPython.core.display.Latex object>

In [3]:
T,W = odesolver.solve(f, y0, (0,1), 100, method = 'Implicit Midpoint')
globalError2=linear.norm(W[:,100]-soln(1)[:,100])
dif=abs(W[:,100]-soln(T)[:,100]) #error global por entrada en t=1
Latex('$h_2=0.01$'+ ls.latexVector(W[:,100],'W_2',form='%f')+ ls.latexVector(dif,'G_2',form='%f') + '$E_{PMImpl}(h_2) = '+str(globalError2)+'$')

<IPython.core.display.Latex object>

El método del Punto Medio Implícito con paso $  \hat{h}  $ tiene un error global de de orden 2 la forma: $$E_{PMImpl}=0\left (\hat{h}^2\right)$$ 

Con $\hat{h}:=h_1=0.1$ obtenemos: $$E_{PMImpl}(h_1)=O\left ( h_1^2 \right ) = O \left( 0.1^2 \right)$$

Con $\hat{h}:=h_2=0.01$ obtenemos: $$E_{PMImpl}(h_2)=O\left ( h_2^2 \right ) = O \left( 0.01^2 \right)$$

Observamos lo siguiente: $$E_{PMImpl}(h_1)=O\left ( h_1^2 \right ) $$ $$  E_{PMImpl}(h_2)=O(h_2^2)$$
$$  h_2= \frac{h_1}{10}   \Rightarrow      E_{PMImpl}(h_2) =O\left(\frac{h_1}{10}\right)^2  $$
$$ \Rightarrow E_{PMImpl}(h_2) \approx \frac{1}{100}O(h_1^2)=\frac{1}{100}E_{PMImpl}(h_1) $$

Esto implica que el error global del método para $h_2$ es aproximadamente 100 veces menor al error global con el paso $h_1$.


In [4]:
Latex('$'+'(0.01)' + 'E_{PMImpl}(h_1)='+str(globalError1/100) + '$')

<IPython.core.display.Latex object>

In [5]:
Latex('$ E_{PMImpl}(h_2)='+str(globalError2) + '$')  

<IPython.core.display.Latex object>

En la operación anterior podemos observar que la reducción del error de $h_1$ a $h_2=\frac{h_1}{10}$es consistente  ya que: $$ E_{PMImpl}(h_2)=8.671073839286167e−06 \approx 8.67819813142771e−06= \frac{1}{100}E_{PMImpl}(h_1) $$

# Shooting Method

# Tomemos el siguiente PVI: 

\begin{equation} 
\label{eq:3}
 \begin{array}{c} y'' = 10 y' (1 - y)\end{array} 
\qquad \mbox{con} \qquad \begin{array}{c}
y(0)=1 \\ y(1) = 1 - pi/20 \end{array} \qquad \mbox{para } t\in [0,1] \qquad 

\mbox{Que podemos reescribir como un sistema de ecuaciones de primer orden de la siguiente manera:} \\
\begin{array}{c} y_1' = y_2 \\
y_2' = 10 y_2 (1 - y_1) \end{array}

\end{equation}



con solución exacta
 está dada por :
\begin{equation}
 \begin{array}{c} y(t) = 1-\frac{\pi}{20} \tan(\frac{\pi}{4}t) \end{array}
\end{equation}


así, definimos f como

In [90]:
f = lambda t, y : np.array([y[1], 10*y[1]*(1-y[0])])

In [240]:
exact = lambda t : 1-pi/20*tan(pi/4*t)

Para aplicar Shooting Method, tomamos una aproximación inicial de y' = 1, o bien,

$\overrightarrow{y}(t)=\left(\begin{array}{c} y_1(t) \\
y_2(t)\end{array} \right) = \left(\begin{array}{c} 1 \\ 1 \right) \end{array}

y utilizaremos el método de Runge Kutta 4 con 1000 pasos para aproximar la función en y(1)

In [241]:
y0 = [1, 0]

In [242]:
T, W = odesolver.solve(f, y0, (0,1), 1000,  method = 'rk4')

In [243]:
ans = W[0][-1]
ans

1.0

Veamos que tan acertada fue nuestra elección

In [245]:
exact(1) == 1-pi/20

True

In [152]:
abs(ans - (1 - pi/20))

0.15707963267948966

Parece ser que nuestra aproximación inicial sobreestimó la solución. Usaremos el método de Newton para seguir generando aproximaciones.

Definimos una función 
$F : \mathbb{R} \rightarrow \mathbb{R}$
que toma una aproximación inical y regresa la distancia 

In [178]:
F = lambda yp0 : odesolver.solve(f, [y0, yp0], I, 1000,  method = 'rk4')[1][0][-1]

In [179]:
I = (0,1)

In [181]:
y0 = 1

In [182]:
F(1)

1.4371120401610173

In [227]:
def shooting(f, y0, yp0, sol, I, m=1000, tol = 1e-7):
    F = lambda yp0 : odesolver.solve(f, [y0, yp0], I, m,  method = 'rk4')[1][0][-1] - sol
    y_init = op.newton(F, yp0, tol = tol, maxiter=7)
    return y_init

In [228]:
# op.newton no puede outputear el número de iteraciones, pero a prueba y error encontramos que toma 7 para converger

In [260]:
y_init = shooting(f, 1, 0, exact(1), (0,1), tol = 1e-10)
#y_init = shooting(f, 1, 0, 1 - pi/20, (0,1), tol = 1e-10)
y_init

-0.12337005501362536

Tomando y_init como aproximación inicial a y'

In [261]:
y0 = [1, y_init]

In [262]:
_, W = odesolver.solve(f, y0, (0,1), 1000,  method = 'rk4')
ans = W[0][-1]

In [263]:
abs(ans - (1 - pi/20))

9.992007221626409e-16

Observamos que con esta aprximación inicial se satisface el BVP